In [6]:
import numpy as np

## Vocabulary

In [3]:
class Vocabulary(object):
    """Class to process text and extract Vocabulary for mapping"""
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
            token_to_idx (dict): a pre-existingmap of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token 
                                for token, idx in self._token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = 1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}
    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)
    def add_token(self, token):
        """Update mapping dicts based on the token.
        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary
        
        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]
    
    def lookup_token(self, token):
        """Retrieve the index associated with the token
        or the UNK index if token isn't present.
        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
            for the UNK functionality
        """
        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
    def lookup_index(self, index):
        """Return the token associated with the index
        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
        KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    def __len__(self):
        return len(self._token_to_idx)

## One Hot encoding Vectorizer

In [7]:
from collections import Counter
import string
class One_hot_Vectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, description_vocab, target_vocab):
        """
        Args:
            description_vocab (Vocabulary): maps words to integers
            target_vocab (Vocabulary): maps class labels to integers
        """
        self.description_vocab = description_vocab
        self.target_vocab = target_vocab
    def vectorize(self, description_tokenized):
        """Create a collapsed one hot vector for the job description
        Args:
            description_tokenized (list): the tokenized job description
        Returns:
            one_hot (np.ndarray): the collapsed onehot encoding
        """
        one_hot = np.zeros(len(self.description_vocab), dtype=np.float32)
        for token in description_tokenized:
            if token not in string.punctuation:
                one_hot[self.description_vocab.lookup_token(token)] = 1
        return one_hot
    @classmethod
    def from_dataframe(cls, description_df, target_cat = True, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        Args:
            description_df (pandas.DataFrame): the description dataset
            cutoff (int): the parameter for frequency based filtering
        Returns:
            an instance of the descriptionVectorizer
        """
        description_vocab = Vocabulary(add_unk=True)
        target_vocab = Vocabulary(add_unk=False)
        # adding category or Job_type
        if target_cat:
            for category in sorted(set(description_df.category)):
                target_vocab.add_token(category)
        
        else:
            
            for job_type in  sorted(set(description_df.job_type_target)):
                target_vocab.add_token(job_type)
        # Add top words if count > provided count
        word_counts = Counter()
        for description in description_df.job_description:
            for word in description:
                if word not in string.punctuation:
                    word_counts[word] += 1
        for word, count in word_counts.items():
            if count > cutoff:
                description_vocab.add_token(word)
        return cls(description_vocab, target_vocab)
    @classmethod
    def from_serializable(cls, contents):
        """Intantiate a descriptionVectorizer from a serializable dictionary
        Args:
            contents (dict): the serializable dictionary
        Returns:
            an instance of the descriptionVectorizer class
        """
        description_vocab = Vocabulary.from_serializable(contents['description_vocab'])
        target_vocab = Vocabulary.from_serializable(contents['target_vocab'])
        return cls(description_vocab=description_vocab, target_vocab=target_vocab)
    def to_serializable(self):
        """Create the serializable dictionary for caching
        Returns:
            contents (dict): the serializable dictionary
        """
        return {'description_vocab': self.description_vocab.to_serializable(),
                'target_vocab': self.target_vocab.to_serializable()}

## Glove Vectorizer

In [8]:
class GloveVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""    
    def __init__(self, description_vocab, target_vocab):
        self.description_vocab = description_vocab
        self.target_vocab = target_vocab

    def vectorize(self, description, vector_length=-1):
        """
        Args:
            description (list) : tokenized description 
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            the vetorized description (numpy.array)
        """
        indices = [self.description_vocab.begin_seq_index]
        indices.extend(self.description_vocab.lookup_token(token) 
                       for token in description.split(" "))
        indices.append(self.description_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.description_vocab.mask_index

        return out_vector

    @classmethod
    def from_dataframe(cls, description_df, target_cat = True, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            description_df (pandas.DataFrame): the target dataset
            cutoff (int): frequency threshold for including in Vocabulary 
        Returns:
            an instance of the descriptionVectorizer
        """
        target_vocab = Vocabulary()
        if target_cat:
            for target in sorted(set(description_df.category)):
                target_vocab.add_token(target)
        
        else:
            for job_type in sorted(set(description_df.job_type_target)):
                target_vocab.add_token(job_type)
            
            
        word_counts = Counter()
        for description in description_df.description:
            for token in description:
                if token not in string.punctuation:
                    word_counts[token] += 1
        
        description_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                description_vocab.add_token(word)
        
        return cls(description_vocab, target_vocab)

    @classmethod
    def from_serializable(cls, contents):
        description_vocab = \
            SequenceVocabulary.from_serializable(contents['description_vocab'])
        target_vocab =  \
            Vocabulary.from_serializable(contents['target_vocab'])

        return cls(description_vocab=description_vocab, target_vocab=target_vocab)

    def to_serializable(self):
        return {'description_vocab': self.description_vocab.to_serializable(),
                'target_vocab': self.target_vocab.to_serializable()}

### Sequence Vocab

In [ ]:
# used for glove encoder
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

How to deal with rare words not in vocab

ASK questions about encoding in TUTE, IE do we have to visualize one hot encoding. Doesn't make too much sense. 
Plus, visulaizing glove is redundant, nothing new.